In [40]:
import pandas as pd
import requests
import numpy as np
import pandas_datareader as pdr
import quandl
import data_load as dl
from pylab import plt, mpl

In [41]:
quandl.ApiConfig.api_key = 'ZuSmYfp_Xc3cJ6a6Zyyz'
sd = '1970-01-01'
ed = '2019-12-31'
fred_recess_inds = ['USARECM' , 'USARECP', 'RECPROUSM156N']
ice_data = ['BAMLCC0A0CMTRIV']
gold_data = ['GOLDPMGBD228NLBM']
yield_inds = ['T10Y2Y']
bbg_data = ['LBUSTRUU']

In [42]:
#Import all data
spx_dy = quandl.get('MULTPL/SP500_DIV_YIELD_MONTH', start_date=sd, end_date=ed)
spx_pr = pd.DataFrame(pdr.DataReader('BA', 'yahoo', start=sd)['Adj Close'])
fred_ind = dl.get_oecd(fred_recess_inds, sd)
ice_tr = pdr.DataReader(ice_data, 'fred', start=sd)
spf_df = dl.get_spf()
inv_ind = dl.get_inv_ind(yield_inds,sd)
gld_df = pdr.DataReader(gold_data, 'fred', start=sd)
bbg_df = pd.read_excel(r'C:\Users\evant\Downloads\recession hindsight.xlsx', sheet_name='Evan', index_col=0)

In [43]:
df = fred_ind.join(spf_df)
df = df.join(spx_pr)
df = df.join(spx_dy)
df = df.join(inv_ind)
df = df.join(ice_tr)
df = df.join(gld_df)
df = df.join(bbg_df)
df = df.fillna(method='ffill').dropna()
df['date'] = df.index
df.reset_index(inplace=True, drop=True)

In [44]:
df['stock returns'] = (df['Adj Close'] - df['Adj Close'].shift()) / df['Adj Close'].shift()
df['bond returns'] = (df[bbg_data] - df[bbg_data].shift()) / df[bbg_data].shift()
df['gold returns'] = (df[gold_data] - df[gold_data].shift()) / df[gold_data].shift()
df.dropna(inplace=True)


## Deterimining the recession indicator

In [45]:
ind = 'recess3'
thresh = 15
stock_allocation = .6
bond_allocation = 1 - stock_allocation

df['allocated_non_recess_return'] = (df['stock returns'] * stock_allocation) + (df['bond returns'] * bond_allocation)
df['recession_ind'] = np.where(df[ind] >= thresh, 1, 0)
df['recession_return'] = np.where(df['recession_ind'] == 1, (df['stock returns'] * bond_allocation) + (df['bond returns'] * stock_allocation), (df['stock returns'] * stock_allocation) + (df['bond returns'] * bond_allocation))

## Plotting

In [9]:
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots

In [10]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df['date'], y=df['stock returns'], name = 'stock returns'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['bond returns'], name = 'bond returns'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['gold returns'], name = 'gold returns'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['allocated_non_recess_return'], name = 'allocated returns'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['recession_return'], name = 'recess allocated returns'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['recession_ind'], name = 'recession indicator'), secondary_y = True)
plot(fig)

'temp-plot.html'

## Growth of 10k

In [46]:
initial_amount = 10000
contribution = 0

In [52]:
df.loc[1, 'stock returns']

-0.024539874808247316

In [47]:
length = list(range(len(df.index)))
ret_cols = ['stock returns', 'bond returns', 'gold returns', 'allocated_non_recess_return', 'recession_return']
growth_cols = ['stock growth', 'bond growth', 'gold growth', 'allocated_growth', 'allocated_recess_growth']

for ret, growth in zip(ret_cols, growth_cols):
    for x in length:
        if x == 0:
            df.loc[x, growth] = initial_amount * (1 + df.loc[x, ret]) 
        else:
            df.loc[x, growth] = (df.loc[x-1, growth] + contribution) *  (1 + df.loc[x, ret])

KeyError: 0

In [13]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=df['date'], y=df['stock growth'], name = 'stock growth'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['bond growth'], name = 'bond growth'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['gold growth'], name = 'gold growth'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['allocated_growth'], name = 'allocated growth'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['allocated_recess_growth'], name = 'recess allocated growth'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['date'], y=df['recession_ind'], name = 'recession indicator'), secondary_y = True)
plot(fig)

'temp-plot.html'

## Efficient Frontier

In [14]:
rets_df = df[['stock returns', 'bond returns', 'gold returns']]
noa = len(rets_df.columns)

In [15]:
def port_ret(weights):
    return np.sum(rets_df.mean() * weights) * 12

In [16]:
def port_vol(weights):
    return np.sqrt(np.dot(weights.T, np.dot(rets_df.cov() * 12, weights)))

In [17]:
allocated_ret = np.sum(df['allocated_non_recess_return'].mean()) * 12
allocated_vol = df['allocated_non_recess_return'].std()*np.sqrt(12)
res_ret = np.sum(df['recession_return'].mean()) * 12
res_vol = df['recession_return'].std()*np.sqrt(12)
gold_ret = np.sum(df['gold returns'].mean()) * 12
gold_vol = df['gold returns'].std()*np.sqrt(12)

In [18]:
prets = []
pvols = []
for p in range (2500):
    weights = np.random.random(noa)
    weights /= np.sum(weights)
    prets.append(port_ret(weights))
    pvols.append(port_vol(weights))
prets = np.array(prets)
pvols = np.array(pvols)

In [19]:
data = go.Scatter(x=pvols, y=prets, mode='markers')
fig = go.Figure(data)
fig.add_trace(go.Scatter(x=[allocated_vol], y=[allocated_ret], mode='markers', name='Static Allocation'))
fig.add_trace(go.Scatter(x=[res_ret], y=[res_vol], mode='markers', name='Recession Allocation'))
#fig.add_trace(go.Scatter(x=[gold_ret], y=[gold_vol], mode='markers', name='Gold Allocation'))
plot(fig)

'temp-plot.html'

## Put it in an app?

In [21]:
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html

In [22]:
# Setup app
app = dash.Dash(__name__)
server = app.server

external_css = ["https://fonts.googleapis.com/css?family=Overpass:300,300i",
                "https://cdn.rawgit.com/plotly/dash-app-stylesheets/dab6f937fd5548cebf4c6dc7e93a10ac438f5efb/dash-technical-charting.css"]

for css in external_css:
    app.css.append_css({"external_url": css})


In [ ]:
app.layout = html.Div(
    [
 html.Div([
            
            html.H1(
                'Cost of Type II Errors in Recession Forecasting',
                className='eight columns',
                style={'text-align': 'center'}
            ),

        ],
            className='row'
        ),
        html.Hr(style={'margin': '0', 'margin-bottom': '5'}),
        
        html.Div([
            html.Div([
                html.Label('Stock fund:'),
                dcc.Dropdown(
                    id='ticker_dropdown',
                    options=tickers,
                    value='SPX',
                ),
            ],
                className='three columns',
            ),
       
        ],
            style={
        'width': '85%',
        'max-width': '1200',
        'margin-left': 'auto',
        'margin-right': 'auto',
        'font-family': 'overpass',
        'background-color': '#F3F3F3',
        'padding': '40',
        'padding-top': '20',
        'padding-bottom': '20'},
)

if __name__ == '__main__':
    app.server.run(debug=False)